<h1 style="color:#333333; text-align:center; line-height: 0;"> <img style="right;" src="logo.png" width=18% height=18%> Reinforcement Learning | Assignment 1 
</h1>
<br/><br/>


The goal of this assignment is to implement:
- value iteration algorithm (35 points)
- policy iteration algorithm (35 points)
- Q-learning algorithm (30 points)

___Total points:___ 100

In [ ]:
%%capture
"""
launch me before you start
"""
from rcognita_framework.pipelines.pipeline_grid_world import action_space

###  <font color="blue"> A brief introduction </font>
Examine it carefully, it covers most of your possible needs to make an assignment.

***

### About Rcognita
The platform for this (and all subsequent work) is [Rcognita](https://gitflic.ru/project/aidynamicaction/rcognita), a framework for applying control theory and machine learning algorithms to control problems, an integral part of which is the closed-loop interaction between the agent under control and the environment evolving over time. In the Rcognita paradigm, the main bearer of all the classes and variables needed to run the simulation is the `pipeline`. 

The main parts of `pipeline` are: 
* `simulator`, which is defined at module `simulators.py` and responsible for simulation of evolution of the environment
* `actor`, defined at module `actors.py`, which is responsible for obtaining of action
* `critic`, defined at module `critics.py`, which is reponsible for learning of reward function and obtaining its value 
* `controller`, which is defined at module `controllers.py` and it's needed to put it all together into an RL (or other) controller
* `system`, which is defined at module `systems.py`. (Possible variations are ODE or discrete, like our grid world)

Other minor things are also declarated in the pipeline and assembled module by module up to the execution of the pipeline itself. 
Just to be on the same page, we provide some notation to prevent further confusions.
* `weights` is the general name and for weights of neural network and for values in tables of value function and policy as well. This agreement comes from the motivation for being consistent with classical RL where critic and actor are being implemented as some neural networks with some **weights**. So, here comes the second term
* `model`. It's obvious that parameters give specificity to something. But the general form itself is being called `model`. There are plenty of models of different types and forms (such as NN). Model is what critic and actor and even running cost always have, no matter what.
* `predictor` - Inspite of it's cryptic name, this object performs an important function, namely, it carries the law by which the dynamics of our system is being predicted in future. For example, if we have some differential equation
$
\begin{cases}
\dot{\boldsymbol x} = \boldsymbol f(\boldsymbol x, \boldsymbol u)\\
\boldsymbol y = h(\boldsymbol x) \\
\boldsymbol x(0)=\boldsymbol x_{0}\\
\end{cases}
$
in general, there are several ways of prediction: 
> - **Analytical**, when we have a precise formula of analytical solution $\boldsymbol x(t)$ to the ODE and have no problems to compute it at any given time. This is great but not that possible in real life. Nevertheless, our predictor could be expressed like:  $\text{predictor}(\boldsymbol x(\tau),dt) = \boldsymbol x(\tau + dt)$
> - **Numerical** way is mostly a case. The simplest way of prediction then is an Euler method:
$\boldsymbol x_{k+1}= \text{predictor}(\boldsymbol x_k, \delta)=\boldsymbol x_{k}+\delta \boldsymbol f\left(\boldsymbol x_{k}, \boldsymbol u_{k}\right) \text {, }$

**Our current grid world case is discrete** and system looks like $\boldsymbol x_{k+1} = \text{predictor}(\boldsymbol x_{k}, \boldsymbol u)=\boldsymbol f(\boldsymbol x_{k}, \boldsymbol u)$. So, **predictor** is trivial here and just invokes a state transition rule, which is already implemented in the system.

***

### About state transition rules
So, now we understand, that predictor is something that predicts the resulting state after our action is applied. But what are the rules? Here are:
* If the agent in the terminal state, it goes nowhere and doesn't receive reward from any action
* If the agent goes out of bounds -> it goes nowhere and just stay on its place and receives reward as usual
* All other cases are intuitive:
    * `predictor.predict((x, y), 0)` -> `(x + 1, y)`
    * `predictor.predict((x, y), 1)` -> `(x - 1, y)`
    * `predictor.predict((x, y), 2)` -> `(x, y + 1)`
    * `predictor.predict((x, y), 3)` -> `(x, y - 1)`
    * `predictor.predict((x, y), 4)` -> `(x, y)`

Now, let's move on the "how-to" part.

***

### "How-to":
* *How to get what i need? Where all classes and variables are stored?* -
Since Rconita is created in a modular style, you almost always can see what's going on in a specific module, going hierarchically from the pipeline to right wherever you need. For example, to get the weights of critic model, here you go:
```python
pipeline.critic.model.weights
```
Or maybe just `critic.model.weights` if `critic` is available as class instance at your scope.

* *How to obtain a prediction correctly?* - 
prediction of the next state can be obtained the following way:
```python
prediction = self.predictor.predict(observation, action)
```
In that case predictor will return a tuple with coordinates of the next state.

* `self.discount_factor` - that's how you can obtain $\gamma$ inside critic and actor
* `self.running_objective(observation, action)` - that's how you can obtain a value $r(\boldsymbol x, \boldsymbol u)$
* *What if i want to use critic inside of actor and vice versa?* - ACTOR IS AWARED OF CRITIC. As critic is awared of actor's model. When you work with actor, you can access **critic's weights** (and vice versa) and objective, don't dublicate your code
* A **full list of parameters** explicitly passed into actor and critic can be found in the testing cell. There is an `initialize_actor_critic()` method, where those classes are instantiated.

There are 3 problems in this assignment in total and many things related to implementation are being behind the scene. Your main objective is to implement 3 algorithms of tables (weights) updating procedure correctly. In case of successful implementation, you will obtain something like that:
<img style="center" src="grid.png" width=58% height=58%>


In [ ]:
"""
Here you go, try it
"""
action_space

In [ ]:
%%capture
"""
Just importing all the necessary stuff here.
DO NOT CHANGE
"""
%matplotlib notebook
%load_ext autoreload
%autoreload 2

from rcognita_framework.pipelines.pipeline_grid_world import PipelineTabular, action_space
from rcognita_framework.rcognita.actors import ActorTabular
from rcognita_framework.rcognita.critics import CriticTabularVI
from rcognita_framework.rcognita.utilities import rc
from copy import deepcopy #wow, this is a helpful easter egg
import warnings

<h2 style="color:#A7BD3F;"> Section 1: Value iteration (VI) algorithm </h2>

***

###  <font color="blue"> 1.1 Value iteration (VI) algorithm mathematical description </font>

The VI algorithm casts into following steps:

I. **Initialization**:
- set iterations number **N_iterations**
- set **discount factor** $\gamma$
- initialize some **value** table $V_0$
- initialize some **policy** table $\rho_0$

II. **Main loop**:<br/>
>**for** i in range(**N_iterations**):
>1. **policy update**:
    >>$\forall \boldsymbol x$ do:
    >>> $\rho_i(x):=\arg \max _{u \in \mathbb{J}}\left\{r(x, u)+\gamma V_i(f(x, u))\right\}$
>    
>2. **value update**:
    >>$\forall \boldsymbol x$ do:
    >>> $V_{i+1}(x):=r\left(x, \rho_i(x)\right)+\gamma V_i\left(f\left(x, \rho_i(x)\right)\right)$

***

#### Your task is to implement main parts of this algorithm - update phases

The following structure is ony a proposition. You can make any implementation you want, even with multiprocessing of whatever, go ahead. The main objective here is to implement correctly the `update` method, because it's being used in the update procedure

In [ ]:
class CriticTabularVIStudent(CriticTabularVI):
    """
    This class is assumed to contain a table which forms the value function.
    Therefore it contains methods for objective calculation and value function updating.
    """
    def update_single_cell(self, observation):
        #############################################
        # YOUR CODE BELOW
        #############################################
        
        #############################################
        # YOUR CODE ABOVE
        #############################################

    def update(self):
        #############################################
        # YOUR CODE BELOW
        #############################################
        
        #############################################
        # YOUR CODE ABOVE
        #############################################

    def objective(self, action, observation):
        #############################################
        # YOUR CODE BELOW
        #############################################
        
        #############################################
        # YOUR CODE ABOVE
        #############################################
    

In [ ]:
class ActorTabularStudent(ActorTabular):
    def update(self):
        #############################################
        # YOUR CODE BELOW
        #############################################
        
        #############################################
        # YOUR CODE ABOVE
        #############################################

###  <font color="blue"> 1.2 Value iteration (VI) testing </font>

In [ ]:
"""
Launch this to try your algorithm in action
"""
class PipelineRLStudent(PipelineTabular):
    def initialize_actor_critic(self):
        self.critic = CriticTabularVIStudent(
            dim_state_space=self.grid_size,
            running_objective=self.running_objective,
            predictor=self.predictor,
            model=self.critic_model,
            actor_model=self.actor_model,
            discount_factor=self.discount_factor,
            terminal_state=self.reward_cell,
        )
        self.actor = ActorTabularStudent(
            dim_world=self.grid_size,
            predictor=self.predictor,
            optimizer=self.actor_optimizer,
            running_objective=self.running_objective,
            model=self.actor_model,
            action_space=action_space,
            critic=self.critic,
            discount_factor=self.discount_factor,
            terminal_state=self.reward_cell,
        )

pipeline_VI = PipelineRLStudent()
pipeline_VI.execute_pipeline()

In [ ]:
"""
LAUNCH THIS ONLY WHEN THE PREVIOUS ANIMATION IS DONE
"""
grade_VI = (abs((pipeline_VI.critic.model.weights[1,1] - 76.) < 1e-2) and 
            (abs(pipeline_VI.critic.model.weights[3,3] + 5.)<1e-2)) * 35

<h2 style="color:#A7BD3F;"> Section 2: Policy iteration </h2>

###  <font color="blue"> Value iteration (VI) algorithm mathematical description </font>

The VI algorithm casts into following steps:

I. **Initialization**:
- set iterations number **N_iterations**
- set **discount factor** $\gamma$
- set **tollerance** for value updating procedure
- set **N_update_iters_max** for value updating procedure to prevent an infiniteness of the loop execution
- initialize some **value** table $V_0$
- initialize some **policy** table $\rho_0$

II. **Main loop**:<br/>
> **for** i in **N_iterations**:
>
> 1. **value update**:
    >>$\forall \boldsymbol x$ do:
    >>> **for** j in range(1, **N_update_iters_max**):
    >>>>
    >>>> $V^{j}_{i}(x) - V^{j-1}_{i}(x) = \text{difference}_j$
    >>>>
    >>>> **if** $\text{difference}_j$ > **tolerance**:
    >>>>> $V^{j}_{i}(x):=r\left(x, \rho_i(x)\right)+\gamma V_i\left(f\left(x, \rho_i(x)\right)\right)$
    >>>>
    >>>> **else**:
    >>>>> **break**
>        
>
> 2. **policy update**:
    >>$\forall \boldsymbol x$ do:
    >>> $\rho_{i+1}(x):=\arg \max _{u \in \mathbb{J}}\left\{r(x, u)+\gamma V_i(f(x, u))\right\}$

***

#### Your task is to implement main parts of this algorithm - update phases

In [ ]:
class CriticTabularPIStudent(CriticTabularVIStudent):
    def __init__(self, *args, tolerance=1e-3, N_update_iters_max=50, **kwargs):
        super().__init__(*args, **kwargs)
        self.tolerance = tolerance
        self.N_update_iters_max = N_update_iters_max

    def update(self):
        #############################################
        # YOUR CODE BELOW
        #############################################
        
        #############################################
        # YOUR CODE ABOVE
        #############################################

###  <font color="blue"> 2.3 Policy iteration (PI) testing </font>

In [ ]:
class PipelineRLStudent(PipelineTabular):
    def initialize_actor_critic(self):
        """
        Uncomment this to go into debug mode. 
        You will be able to see a full traceback under the static picture of grid.
        """
        #self.no_visual = True
        self.critic = CriticTabularPIStudent(
            dim_state_space=self.grid_size,
            running_objective=self.running_objective,
            predictor=self.predictor,
            model=self.critic_model,
            actor_model=self.actor_model,
            discount_factor=self.discount_factor,
            terminal_state=self.reward_cell,
        )
        self.actor = ActorTabularStudent(
            dim_world=self.grid_size,
            predictor=self.predictor,
            optimizer=self.actor_optimizer,
            running_objective=self.running_objective,
            model=self.actor_model,
            action_space=action_space,
            critic=self.critic,
            discount_factor=self.discount_factor,
            terminal_state=self.reward_cell,
        )

pipeline_PI = PipelineRLStudent()
pipeline_PI.execute_pipeline()

In [ ]:
"""
LAUNCH THIS ONLY WHEN THE PREVIOUS ANIMATION IS DONE
"""
grade_PI = (abs((pipeline_PI.critic.model.weights[1,1] - 76.) < 1e-2) and 
            (abs(pipeline_PI.critic.model.weights[3,3] + 5.)<1e-2)) * 35

<h2 style="color:#A7BD3F;"> Section 3: Q-learning </h2>

###  <font color="blue"> 3.1 Q-learning (QL) algorithm mathematical description </font>

The QL algorithm casts into following steps:

I. **Initialization**:
- set iterations number **N_iterations**
- set **discount factor** $\gamma$
- initialize some **Q** table $Q_0$
- initialize some **policy** table $\rho_0$

II. **Main loop**:<br/>
>**for** i in range(**N_iterations**):
>
> 1. **policy update**:
    >>$\forall \boldsymbol x$ do:
    >>> $\rho_i(x): =\arg \max _{u \in J} Q_i(x, u)$
>    
> 2. **Q-function update**:
    >>$\forall \boldsymbol x$ do:
    >>> $Q_{i+1}(x, u): =r(x, u)+\gamma Q_i\left(f(x, u), \rho_i(f(x, u))\right)$

#### Your task is to implement main parts of this algorithm - update phases

In [ ]:
class CriticTabularQLStudent(CriticTabularVI):
    """
    This class is assumed to contain a table which forms the Q-function.
    Therefore it contains methods for objective calculation and Q-function updating.
    """
    def update_single_cell(self, observation):
        #############################################
        # YOUR CODE BELOW
        #############################################
        
        #############################################
        # YOUR CODE ABOVE
        #############################################

    def update(self):
        #############################################
        # YOUR CODE BELOW
        #############################################
        
        #############################################
        # YOUR CODE ABOVE
        #############################################

    def objective(self, observation, action):
        #############################################
        # YOUR CODE BELOW
        #############################################
        
        #############################################
        # YOUR CODE ABOVE
        #############################################

###  <font color="blue"> 3.2 Q-learning (QL) testing </font>

In [ ]:
class PipelineRLStudent(PipelineTabular):
    def initialize_actor_critic(self):
        """
        Uncomment this to go into debug mode. 
        You will be able to see a full traceback under the static picture of grid.
        """
        #self.no_visual = True
        self.critic = CriticTabularQLStudent(
            dim_state_space=self.grid_size,
            running_objective=self.running_objective,
            predictor=self.predictor,
            model=self.critic_model,
            actor_model=self.actor_model,
            discount_factor=self.discount_factor,
            terminal_state=self.reward_cell,
        )
        self.actor = ActorTabularStudent(
            dim_world=self.grid_size,
            predictor=self.predictor,
            optimizer=self.actor_optimizer,
            running_objective=self.running_objective,
            model=self.actor_model,
            action_space=action_space,
            critic=self.critic,
            discount_factor=self.discount_factor,
            terminal_state=self.reward_cell,
        )

pipeline_QL = PipelineRLStudent()
pipeline_QL.execute_pipeline()

In [ ]:
"""
LAUNCH THIS ONLY WHEN THE PREVIOUS ANIMATION IS DONE
"""
grade_QL = (abs((pipeline_QL.critic.model.weights[1,1] - 76.) < 1e-2) and 
            (abs(pipeline_QL.critic.model.weights[3,3] + 5.)<1e-2)) * 30

In [ ]:
"""
Here are your expected total points
"""
print(f"Your total grade: {grade_VI+grade_PI+grade_QL}")

In case if you sruggle with your assignment, just PM him on telegram 😊 -> @odinmaniac